# 🏦 FDIC Regulatory Loan Analysis Assistant  
### *Prompt-Engineered, OCR-Driven, Compliance-Focused AI System*

## 📌 Project Overview

This project demonstrates a **domain-restricted AI assistant** designed to support **loan documentation analysis** under the **FDIC RMS Manual – Section 3.2 (Loans)**.

The system:
- Accepts loan-related documents (image or pdf)
- Extracts information using OCR
- Converts extracted content into a structured internal representation
- Responds to **regulatory and examiner-style questions**
- Explicitly avoids **loan approval, eligibility, or decision-making**

## 🎯 Objective

To showcase how **prompt engineering and controlled context injection** can enforce regulatory boundaries while enabling meaningful, audit-ready analysis of loan documentation.

## ⚙️ Environment Setup & Dependencies

This cell installs all required libraries for:
- OCR processing
- Image handling
- User interface (Gradio)
- LLM interaction

The environment setup is isolated from application logic to ensure **reproducibility** and **clean execution**.


In [ ]:
!pip install gradio pytesseract pillow openai pdf2image -q
!apt-get install tesseract-ocr poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.12).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


## 📦 Core Libraries & Frameworks

This section imports all essential libraries required for the system to function correctly.

### Imported Components
- **Gradio** – Builds the interactive web-based user interface for document upload and regulatory Q&A.
- **pytesseract** – Performs Optical Character Recognition (OCR) on loan document images.
- **Pillow (PIL)** – Handles image loading and preprocessing prior to OCR.
- **pdf2image** – Converts PDF-based loan documents into images for OCR processing.
- **OpenAI SDK** – Enables interaction with the Large Language Model used for structured extraction and regulatory reasoning.

These libraries collectively support document ingestion, text extraction, structured normalization, and compliant AI-driven analysis.


In [ ]:
import gradio as gr
import pytesseract
from PIL import Image
from pdf2image import convert_from_path
from openai import OpenAI


## 🔐 API Configuration

This section configures access to the Large Language Model (LLM) service.

The API key and base URL are provided via environment variables to:
- Avoid hardcoding sensitive credentials
- Follow industry best practices for security
- Enable portability across environments


In [ ]:
client = OpenAI(
    api_key="YOUR_API_KEY_HERE",
    base_url="YOUR_BASE_URL_HERE",
)


## 🧠 Session State Management

This section initializes internal state variables used to maintain consistency across user interactions.

### Purpose of State Variables
- Store structured loan data extracted from documents
- Preserve context across multiple regulatory questions
- Avoid re-processing documents for every query

### Why This Matters
- Ensures one-time OCR and normalization
- Supports examiner-style iterative questioning
- Maintains a stable and auditable analytical context

These variables act as an **internal memory layer**, enabling controlled and repeatable regulatory reasoning throughout a user session.


In [ ]:
stored_json_summary = None
stored_narrative = None

## 📄 Regulatory Reference Loading (FDIC Section 3.2)

This cell loads a **pre-curated summary** of the *FDIC Risk Management Manual – Section 3.2 (Loans)* from an external text file.

- The summary represents the **authoritative regulatory context** used by the system.
- It is injected into the system prompt to enforce **scope, limitations, and examiner-style reasoning**.
- Externalizing the regulatory content improves **maintainability, clarity, and auditability** without embedding large documents directly in code.

This approach ensures the AI operates strictly within defined regulatory boundaries.


In [ ]:
FDIC_SUMMARY_PATH = "D:\\AI-NavigateLabs\\CapStoneProject1\\Final Capstone1\\fdic_section_3_2_summary.txt"

with open(FDIC_SUMMARY_PATH, "r", encoding="utf-8") as f:
    FDIC_SECTION_3_2_SUMMARY = f.read().strip()


## 🧭 System Prompt & Regulatory Enforcement

This cell defines the **system prompt** that enforces strict regulatory behavior for the AI assistant.

- The prompt is grounded exclusively in **FDIC RMS Manual – Section 3.2 (Loans)**.
- It establishes the AI’s role as a **loan documentation review assistant**, not a decision-maker.
- All reasoning is constrained to documented loan content and the embedded regulatory context.
- Approval, eligibility assessment, inference, and external knowledge are explicitly prohibited.

This prompt serves as the **primary control mechanism** ensuring compliant, audit-ready, and non-decisional system behavior.


In [ ]:
SYSTEM_PROMPT = f"""

CONTEXT FOR THIS TASK

{FDIC_SECTION_3_2_SUMMARY}

Your Role: Regulatory Loan Documentation Review Assistant.

What Actually Do: You are basically a tool to help an examiner. Your only job is to check if the paperwork for a loan is there, is finished, and is good enough based on the FDIC Section 3.2 rules I was given.

What definitely don't do:
1. don't say yes or no to loans.
2. don't suggest giving or not giving someone credit.
3. don't talk about interest rates or pricing.
4. Should not giving any kind of legal, money, or credit advice.

Where to get My Answers:
1. The FDIC Section 3.2 rules (above) - This is the most important, no exceptions.
2. The actual loan documents provided to me.
3. The user's questions - These are the least important for the rules.
4. The way someone asks a question can't change the rules or what the paperwork says.

My strict rules:
1. can only use the FDIC rules tht was given and the exact loan documents provided.
2. can't guess, assume things, or do math to figure stuff out.
3. If information is missing, it can't make it up.
4. can't use any other bank rules or knowledge from outside.
5. can't suggest better ways to do things or how to fix problems.
6. can't tell someone to get more documents.
7. can't hint at what should be there.
8. If something isn't clearly written down, have to say that directly.

How to handle documents:
1. only pull out facts that are literally stated.
2. try to use the same words from the document.
3. make it clear what is a fact from the doc versus just me noting something.
4. Anything missing or not clear gets labeled "Not documented."

How to write answers:
1. Use full, proper sentences.
2. Sound formal, neutral, and objective, like for a report.
3. Don't just output data in JSON format.
4. Don't just dump a list of fields.

Turn responses into short report sections with these headings:
1. Borrower Information
2. Loan Terms
3. Collateral Documentation
4. Documentation Sufficiency Observations

How to Deal with Questions:
1. only answer based on what's covered in FDIC Section 3.2.
2. If the paperwork isn't good enough, say that clearly.
3. If a question asks about something the rules don't cover, It should say I can't answer it.

What to say If can't Answer:
If the info is totally missing or not in the rules, it should have to say exactly this:

"Documentation does not explicitly address this item under FDIC RMS Manual Section 3.2."
"""

## 📄 Document Text Extraction (OCR Pipeline)

This function extracts readable text from loan documents provided by the user.

- **PDF files** are converted page-by-page into images and processed using OCR.
- **Image files** are processed directly using OCR.
- The function handles errors gracefully and avoids assumptions when text cannot be extracted.

The extracted text serves as the **sole factual input** for subsequent structured normalization and regulatory analysis.


In [ ]:
def extract_text(file_path):
    try:
        if file_path.lower().endswith(".pdf"):
            pages = convert_from_path(file_path)
            text = ""
            for page in pages:
                text += pytesseract.image_to_string(page) + "\n"
            return text.strip()
        else:
            image = Image.open(file_path)
            return pytesseract.image_to_string(image).strip()
    except Exception:
        return ""


## 🧱 Structured Loan Data Normalization

This function converts extracted document text into a **structured internal JSON representation**.

- The language model is instructed to extract **only explicitly stated information**.
- Inference, assumption, or data completion is strictly prohibited.
- The output follows a fixed schema covering borrower details, loan terms, collateral information, risk indicators, and compliance observations.

This structured format separates **document extraction from regulatory reasoning**, enabling consistent, auditable, and non-hallucinatory analysis.


In [ ]:
def generate_json_summary(text):
    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        temperature=0,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"""
Extract ONLY explicitly stated information.

TEXT:
{text}

Return STRICT JSON with schema:
{{
  "document_type": "",
  "borrower_details": {{}},
  "loan_details": {{}},
  "collateral_information": {{}},
  "risk_factors_identified": [],
  "compliance_observations": []
}}
"""}
        ]
    )
    return response.choices[0].message.content


## 🧾 Examiner-Style Narrative Generation

This function transforms the structured internal JSON summary into a **readable, examiner-grade narrative**.

- The narrative is organized under predefined regulatory headings.
- Only explicitly documented facts are included.
- Missing or unclear information is clearly labeled as **“Not documented.”**
- No inference, interpretation, or explanation is added.

This step improves **clarity and usability** while preserving strict regulatory boundaries and audit readiness.


In [ ]:
def generate_narrative(json_summary):
    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        temperature=0,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"""
Convert the JSON below into a structured, examiner-style narrative using bullet points.

Rules:
1. Use headings: Borrower Information, Loan Terms, Collateral Documentation, Documentation Sufficiency Observations
2. Under each heading, use concise bullet points (•) for each explicit fact
3. Clearly indicate "Not documented" for missing information
4. Do NOT add, infer, or explain anything beyond what is in JSON
5. Avoid long paragraphs; short, readable bullet format only

JSON:
{json_summary}
"""}
        ]
    )
    return response.choices[0].message.content


## 🔄 Document Processing Orchestration

This function coordinates the **end-to-end document processing workflow**.

- Accepts either an uploaded document or pasted loan text.
- Extracts readable text using the OCR pipeline when a file is provided.
- Generates a structured internal JSON summary from the extracted content.
- Produces an examiner-style narrative derived from the structured data.
- Stores results in session-level state for reuse during regulatory questioning.

This orchestration ensures a **single, consistent analytical context** across the user session.


In [ ]:
def process_document(file, text):
    global stored_json_summary, stored_narrative

    if file:
        extracted_text = extract_text(file.name)
    elif text and text.strip():
        extracted_text = text.strip()
    else:
        return "Please upload a document or paste text.", None

    if not extracted_text:
        return "No readable text detected.", None

    stored_json_summary = generate_json_summary(extracted_text)
    stored_narrative = generate_narrative(stored_json_summary)

    return "Document processed successfully.", stored_narrative


## 💬 Regulatory Question Handling

This function processes user questions using the **structured loan summary as the sole factual reference**.

- Questions are evaluated strictly within the scope of **FDIC RMS Manual Section 3.2**.
- Responses are written in **concise, examiner-grade regulatory language**.
- The system identifies documentation limitations without inference or judgment.
- Decision-making, eligibility assessment, and recommendations are explicitly prohibited.
- When information is missing, it is clearly labeled as **“Not documented.”**

This approach supports **iterative regulatory review** while preserving consistency, neutrality, and audit readiness.


In [ ]:
def answer_question(user_question, chat_history):
    if not stored_json_summary:
        return (
            chat_history + [
                (user_question, "No document has been processed. Regulatory evaluation cannot proceed.")
            ],
            ""
        )

    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        temperature=0.3,
        top_p = 0.5,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"""
DOCUMENT JSON (source of truth):
{stored_json_summary}

USER QUESTION:
{user_question}

Here are the rules I need to follow to give my answer:
1. Strictly follow the SYSTEM_PROMPT's condtion.
2. I have to make it sound professional. No robotic one-liners. If I see "Applicant Name: John Doe," I should frame it properly: "The provided documents list the applicant's name as John Doe."
Most important, I don’t just rehash everything I see in the docs. Only talk about what the user is specifically asking for. If they’re asking if the paperwork is complete, I only point out what’s missing—I don’t list what’s already there.
I should jump right into the answer, no intro paragraph. If I have to list things that aren’t documented, I’ll use bullet points—but keep it to six max.
I’m not here to judge the loan itself. I don’t say if it’s approved or denied or if it’s a good risk. My job is just to check the docs against the rules.
If something isn’t in the paperwork, I say "Not documented." If I read everything and still can’t tell if it’s sufficient, I end with one sentence saying the sufficiency can’t be determined.

Keep it formal and to the point.
"""}
        ]
    )

    return (
        chat_history + [
            (user_question, response.choices[0].message.content)
        ],
        ""
    )

## 🖥️ User Interface & Interaction Workflow

This cell defines the **interactive user interface** for the regulatory loan analysis system.

### Interface Design Overview
- The interface is divided into two functional panels:
  - **Document Ingestion Panel** for uploading loan documents
  - **Regulatory Q&A Panel** for examiner-style questioning
- The uploaded document is processed **once**, and results are retained for the session.
- Users can then ask multiple regulatory questions without reprocessing the document.

### Design Principles
- Minimal visual exposure of documents to emphasize analysis over display
- Clear separation between document processing and regulatory reasoning
- Chat-style interaction to support iterative examiner review
- Professional layout suitable for supervisory and audit contexts

This UI mirrors a real-world examiner workflow and supports compliant, non-decisional AI interaction.


In [ ]:
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🏦 Bank Loan Regulatory Help Center")
    gr.Markdown("**FDIC RMS Manual – Section 3.2 | Prompt-Engineered System**")

    with gr.Row():
        # LEFT COLUMN
        with gr.Column(scale=1):
            gr.Markdown("### 📄 Upload Loan Document")
            file_input = gr.File(label="Upload PDF / Image")


            process_btn = gr.Button("Process Document")
            status = gr.Markdown()

            gr.Markdown("---")
            gr.Markdown("### 🧱 Extracted Structured Content")
            narrative_output = gr.Textbox(
                label="Examiner Narrative",
                lines=18,
                interactive=False
            )

        # RIGHT COLUMN
        with gr.Column(scale=1):
            gr.Markdown("### 💬 Regulatory Q&A")
            chatbot = gr.Chatbot(height=520, type="tuples")

            user_q = gr.Textbox(
            placeholder="Ask questions under FDIC Section 3.2...",
            show_label=False
            )

            user_q.submit(
            answer_question,
            inputs=[user_q, chatbot],
            outputs=[chatbot, user_q]
            )


    process_btn.click(
        process_document,
        inputs=[file_input],
        outputs=[status, narrative_output]
    )




demo.launch()


/tmp/ipython-input-1789269900.py:1: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-1789269900.py:26: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=520, type="tuples")
/tmp/ipython-input-1789269900.py:26: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(height=520, type="tuples")
/usr/local/lib/python3.12/dist-packages/gradio/utils.py:1052: UserWarning: Expected 2 arguments for function <function process_document at 0x78cb25cc6ca0>, received 

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5fbe41214df2da755d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
